***
 #  Examples Using `pandas`, `np.fft`, and `scipy.optimize`
 ***
 ## <font color="purple">Demo: use `pandas` to process large CSV data set</font>


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

from IPython.display import display, HTML, Audio

### NOAA data for station KCOS (Colorado Springs Airport)

In [ ]:
Audio("http://quizdog.com/numpy/kcos_atis.wav")

### initial look at the data
+ `pandas` returns data as a `DataFrame`
+ a `DataFrame` contains multiple `Series`,
* underlying each `Series` is an `ndarray`

In [ ]:
URL = "http://quizdog.com/numpy/kcos_temperature.csv"
data = pd.read_csv(URL)           # read URL into a DataFrame

print ("\nData Types")
print(data.dtypes)

print ("\nData Sample (Note: Some Missing Temperature Values (NaN)")
display(data[16412:16418])

print ("\nData Description")
print(data.describe())

In [ ]:
print ("\nData Sample (Note: Weather Observations)")
display(data[110:120])

### filter out everything except hourly observations (at 54 after the hour)

In [ ]:
data = pd.read_csv(URL)
#data = pd.read_csv(URL, converters = {'date' : pd.to_datetime })

# use Boolean indexing to select only rows where timestamp is 54 minutes after the hour

hourly = data[ [timestamp.split(':')[1] == '54' for timestamp in data.date ] ]
#hourly = data[ [timestamp.minute == 54 for timestamp in data] ]

print ("number of hourly observations:", len(hourly))
print ("number of days:", len(hourly) / 24)


### some hourly observations are missing

In [ ]:
null = hourly.temp.isnull()           # True where value in temp Series is NaN (not a number) 
where = hourly.temp[null].index       # find the indices of the True values

print("temperatures missing at:")
display(hourly.loc[where[0:2]])       # look at first two missing temperatures


### interpolate to fill in the missing temperature values

In [ ]:
hourly = hourly.interpolate()                      # interpolate missing values

print("interpolated temperatures:")
display(hourly.loc[where[0:2]])


### plot the hourly temperature

In [ ]:
temp = hourly.temp.values                           # get underlying ndarray for temp

average_temp = np.average(temp)                     # note: average is a universal function
print("Average Temperatue: {0:.2f}\N{DEGREE SIGN}C".format(average_temp))

plt.title("Hourly Temperature \N{DEGREE SIGN}C")
plt.ylabel("Temperature \N{DEGREE SIGN}C")
plt.xlabel("Hours Since 1/1/2007")

plt.plot(temp, 'c')                                 # plot temp data in cyan
plt.plot(np.ones(len(temp)) * average_temp, 'r')    # draw line for average temp in red


# <font color="purple">Demo: Fourier Transform with `np.fft.fft`</font>

In [ ]:
ft = np.fft.fft(temp)
ft = ft[:len(ft)//2]
ft = abs(ft)

In [ ]:
plt.subplot(211)
plt.axis([-5, 50, 0, 1000000])
plt.xlabel("frequency (cycles / 10 years)")
plt.plot(ft, 'r')

plt.subplot(212)
plt.axis([-100, 5000, 0, 1000000])
plt.xlabel("frequency (cycles / 10 years)")
plt.plot(ft, 'r')

plt.tight_layout()

In [ ]:
print("peaks at", np.where(ft > 100000)[0], "cycles")

`0     'DC' component (average temp != 0)
10    10 cycles / 10 years = 1 cycle / year
3653  3653 cycles / 10 years = 365.3 cycles / year  (note remember the 3 leap days)`

## <font color="purple">Demo: modify `shape` to to find average daily temperature</font>
+ change shape to array of arrays
+ each inner array has temperatures for one day

In [ ]:
np.average(temp[0:24])              # find average temperature of first 24 elements (ie. day 1)

In [ ]:
temp.shape = (-1, 24)               # reshape into an array of 24 element arrays (ie. a given day)
np.average(temp[0])                 # verify first average daily temperature gives same result

In [ ]:
daily = np.average(temp, axis=1)    # vectorized average over second axis (the day)

plt.title("Average Daily Temperature \N{DEGREE SIGN}C")
plt.ylabel("Temperature \N{DEGREE SIGN}C")
plt.xlabel("Days Since 1/1/2007")
plt.plot(daily, 'c')

In [ ]:
ft = np.fft.fft(daily)
ft = ft[:len(ft)//2]
ft = abs(ft)
plt.axis([-5, 50, 0, 50000])
plt.plot(ft, 'r')

In [ ]:
print("peaks at", np.where(ft > 20000)[0], "cycles")

## <font color="purple">Demo: Using `scipy.optimize.leastsq` to model data with a function</font>

## model the daily average temperature with a 4 parameter sinusoide

In [ ]:
def model(t, A, freq, phase, offset):
    """model function to be fitted to data"""
    return A * np.sin(freq * t + phase) + offset

first_guess = (15, 1/55, np.pi/2, 10)   # first guess for (A, freq, phase, offset) to fit data

t = np.arange(0, len(daily))

plt.title("Average Daily Temperature \N{DEGREE SIGN}C \nvs Model (Initial Guess)")
plt.ylabel("Temperature \N{DEGREE SIGN}C")
plt.xlabel("Days Since 1/1/2007")
plt.plot(t, daily, 'c')
plt.plot(model(t, *first_guess), 'b')

## define error function then minimize it to better fit model to data

In [ ]:
def error(parameters, data, t):
    """compute the difference between the data
        and the model function at given time t"""
    A, freq, phase, offset = parameters
    return data - model(t, A, freq, phase, offset)

from scipy.optimize import leastsq

parameters = leastsq(error, first_guess, args=(daily, t))

In [ ]:
print ("best fit parameters are:", parameters)


In [ ]:
plt.title("Average Daily Temperature \N{DEGREE SIGN}C\nvs Model (Best Fit)")
plt.ylabel("Temperature \N{DEGREE SIGN}C")
plt.xlabel("Days Since 1/1/2007")
plt.plot(t, daily,'c')
plt.plot(t, model(t, *parameters[0]), 'b')